In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from mxnet import autograd, gluon, init, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
import random
import numpy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from mpl_toolkits.axes_grid1 import make_axes_locatable

npx.set_np()

In [ ]:
## 数据文件的位置！！
DXX=pd.read_table(r"E:\NN\Data\1Wire\1Wire_XX.txt", sep='\s+', header=None)
DYY=pd.read_table(r"E:\NN\Data\1Wire\1Wire_YY.txt", sep='\s+', header=None)

In [ ]:
NXX=np.array(DXX.values, dtype=np.float32, ctx=npx.gpu()).reshape(10000,4,1,101)
YY=DYY

In [ ]:
n_train = 8000
train_features = NXX[:n_train,:,:,:]
test_features = NXX[n_train:,:,:,:]
train_labels = np.array(YY[:n_train].values*100, dtype=np.float32, ctx=npx.gpu())
test_labels = np.array(YY[n_train:].values*100, dtype=np.float32, ctx=npx.gpu())

In [ ]:
batch_size=128
train_iter = d2l.load_array((train_features, train_labels), batch_size)
test_iter = d2l.load_array((test_features, test_labels), batch_size)

In [ ]:
def MSE(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += ((net(X)-y)**2).sum()
        n += y.size
    return acc_sum / n

In [ ]:
net = nn.Sequential()
net.add(nn.BatchNorm(),
        nn.Conv2D(channels=3, kernel_size=3, padding=1, activation='relu'),
        nn.Dense(202, activation='relu'),
        nn.Dense(101))
net.initialize(init.Normal(sigma=0.01), ctx=npx.gpu())
# net.load_parameters("1Wire_23", ctx=npx.gpu()) ## 这是可以直接调用训练好的参数！！需要注释前一行对参数的初始化
loss = gluon.loss.L2Loss()

In [ ]:
train_ls, test_ls = [], []

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.00001,})

In [ ]:
old_epochs=0
new_epochs=10

for epoch in range(new_epochs):
    for X, y in train_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    train_ls.append(MSE(train_iter, net))
    test_ls.append(MSE(test_iter, net))
    print('epoch %d, train MSE %.6f, test MSE %.6f' % (old_epochs+epoch+1, train_ls[-1], test_ls[-1]))

In [ ]:
plt.plot(np.arange(old_epochs+new_epochs),train_ls,label='train data')
plt.plot(np.arange(old_epochs+new_epochs),test_ls,label='test data')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.yscale('log')
plt.legend()

In [ ]:
# net.save_parameters("1Wire_23")  ## 这是可以保存训练好的参数！！

In [ ]:
def plots(net,iindex=0):
    if iindex==0:
        ii=random.randint(n_train,DXX.shape[0])
    else:
        ii=iindex
    px=NXX[ii,:,:,:]
    py=YY.iloc[ii].values
    lx=DXX.iloc[ii,0:101].values
    nxx=px.copyto(npx.gpu()).reshape(1,4,1,101)
    nyy=net(nxx).reshape(101,)

    plt.plot(lx,np.linspace(0,0,101),color='black')
    plt.bar(lx,DXX.iloc[ii,101*2:101*3].values,color='g',width =.8) 
    plt.plot(lx,-py*100,color='r',label='FEM',linestyle='--')
    plt.plot(lx,-nyy,color='b',label='NET')
    plt.legend()

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(2, 2, 1); plots(net)
plt.subplot(2, 2, 2); plots(net)
plt.subplot(2, 2, 3); plots(net)
plt.subplot(2, 2, 4); plots(net)

In [ ]:
sMAPEerror=[]
for i in range(8000,10000):
    px=NXX[i,:,:,:]
    py=YY.iloc[i].values*100

    nxx=px.reshape(1,4,1,101)
    nyy=net(nxx).reshape(101,)

    ny=nyy.asnumpy()
    tmp=(abs(py-ny)/(abs(py)+abs(ny))).sum()
    tmp=tmp/py.size
    sMAPEerror.append(tmp)
print('Maxerror:', max(sMAPEerror))
print('Maxindex:', 8000+sMAPEerror.index(max(sMAPEerror)))

In [ ]:
sum(sMAPEerror)/2000